In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Dict, List, Any
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import json
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
llm_key = os.getenv('llm_key')

model = ChatOpenAI(
    model="openai/gpt-oss-20b:free",
    api_key=llm_key,
    base_url="https://openrouter.ai/api/v1",
    temperature=0.7
)

In [4]:
class ModificationState(TypedDict):
    original_workflow: Dict[str, Any]
    modification_request: str
    analysis: Dict[str, Any]
    clarifying_questions: List[Dict[str, Any]]
    user_answers: Dict[str, Any]
    modification_plan: Dict[str, Any]
    modified_workflow: Dict[str, Any]
    validation_result: Dict[str, Any]
    iteration_count: int
    changes_applied: List[str]

In [5]:
def load_existing_workflow(state: ModificationState):
    """
    Load and analyze the existing workflow
    """
    
    original_workflow = state.get("original_workflow", {})
    
    print("\n📂 LOADING EXISTING WORKFLOW")
    print("="*80)
    
    metadata = original_workflow.get("metadata", {})
    workflow_name = metadata.get("workflow_name", "Unknown")
    
    print(f"\n✓ Workflow: {workflow_name}")
    print(f"✓ Description: {metadata.get('description', 'N/A')}")
    
    # Analyze current structure
    approval_chain = original_workflow.get("workflow_analysis", {}).get("approval_chain", [])
    form_questions = original_workflow.get("microsoft_forms", {}).get("questions", [])
    workflow_steps = original_workflow.get("power_automate_workflow", {}).get("steps", [])
    
    print(f"✓ Approval Levels: {len(approval_chain)}")
    print(f"✓ Form Questions: {len(form_questions)}")
    print(f"✓ Workflow Steps: {len(workflow_steps)}")
    
    print("\n📋 Current Approval Chain:")
    for approver in approval_chain:
        print(f"   Level {approver['level']}: {approver['approver_role']}")
    
    return {
        "original_workflow": original_workflow,
        "iteration_count": 0,
        "changes_applied": []
    }


In [6]:
def analyze_modification_request(state: ModificationState):
    """
    Use LLM to analyze what needs to be modified
    """
    
    original_workflow = state.get("original_workflow", {})
    modification_request = state.get("modification_request", "")
    
    print("\n🤖 ANALYZING MODIFICATION REQUEST")
    print("="*80)
    
    prompt = f"""
You are a workflow modification expert. Analyze what needs to be changed in this workflow.

CURRENT WORKFLOW:
{json.dumps(original_workflow, indent=2)}

USER'S MODIFICATION REQUEST:
{modification_request}

Analyze and return ONLY valid JSON:
{{
  "modification_type": "add_approver|remove_approver|change_sequence|add_field|modify_notification|add_condition|other",
  "affected_components": ["approval_chain", "form_schema", "excel_schema", "workflow_steps"],
  "complexity": "simple|moderate|complex",
  "changes_needed": [
    {{
      "component": "approval_chain",
      "action": "add|remove|modify|reorder",
      "details": "specific description of change"
    }}
  ],
  "potential_conflicts": ["list any issues"],
  "requires_clarification": true/false,
  "summary": "Brief summary of what will be changed"
}}

Return ONLY the JSON, no other text.
"""

    try:
        messages = [
            SystemMessage(content="You are a workflow analysis expert. Always return valid JSON only."),
            HumanMessage(content=prompt)
        ]
        
        response = model.invoke(messages)
        response_text = response.content.strip()
        
        # Clean response
        if response_text.startswith("```json"):
            response_text = response_text.replace("```json", "").replace("```", "").strip()
        
        analysis = json.loads(response_text)
        
        print(f"\n✓ Modification Type: {analysis.get('modification_type', 'unknown')}")
        print(f"✓ Complexity: {analysis.get('complexity', 'unknown')}")
        print(f"✓ Components Affected: {', '.join(analysis.get('affected_components', []))}")
        print(f"\n📝 Summary: {analysis.get('summary', 'N/A')}")
        
        return {"analysis": analysis}
        
    except Exception as e:
        print(f"\n⚠️  Analysis failed: {e}")
        return {
            "analysis": {
                "modification_type": "other",
                "affected_components": ["unknown"],
                "complexity": "moderate",
                "changes_needed": [],
                "requires_clarification": True,
                "summary": "Manual analysis required"
            }
        }

In [7]:
def generate_clarifying_questions(state: ModificationState):
    """
    Generate questions if clarification needed
    """
    
    analysis = state.get("analysis", {})
    
    if not analysis.get("requires_clarification", False):
        print("\n✓ No clarification needed, proceeding with modifications")
        return {"clarifying_questions": []}
    
    print("\n🤖 GENERATING CLARIFYING QUESTIONS")
    print("="*80)
    
    modification_request = state.get("modification_request", "")
    original_workflow = state.get("original_workflow", {})
    
    prompt = f"""
You need to clarify the user's modification request.

MODIFICATION REQUEST: {modification_request}

ANALYSIS: {json.dumps(analysis, indent=2)}

CURRENT WORKFLOW STRUCTURE:
- Approval Chain: {len(original_workflow.get('workflow_analysis', {}).get('approval_chain', []))} levels
- Form Fields: {len(original_workflow.get('microsoft_forms', {}).get('questions', []))} questions

Generate 2-4 specific clarifying questions. Return ONLY valid JSON:
{{
  "questions": [
    {{
      "id": "q1",
      "question": "Where should this new approver be inserted in the chain?",
      "type": "choice",
      "options": ["Before Mentor", "After Mentor", "At the end"],
      "required": true
    }},
    {{
      "id": "q2",
      "question": "What should happen if this approver rejects?",
      "type": "text",
      "required": false
    }}
  ]
}}

Return ONLY the JSON.
"""

    try:
        messages = [
            SystemMessage(content="You generate clarifying questions. Always return valid JSON only."),
            HumanMessage(content=prompt)
        ]
        
        response = model.invoke(messages)
        response_text = response.content.strip()
        
        if response_text.startswith("```json"):
            response_text = response_text.replace("```json", "").replace("```", "").strip()
        
        questions_data = json.loads(response_text)
        questions = questions_data.get("questions", [])
        
        print(f"\n✓ Generated {len(questions)} questions")
        
        return {"clarifying_questions": questions}
        
    except Exception as e:
        print(f"\n⚠️  Question generation failed: {e}")
        return {"clarifying_questions": []}


In [8]:
def collect_clarifications(state: ModificationState):
    """
    Collect clarifying answers from user
    """
    
    questions = state.get("clarifying_questions", [])
    
    if not questions:
        return {"user_answers": {}}
    
    print("\n📝 CLARIFYING QUESTIONS")
    print("="*80)
    
    answers = {}
    
    for question in questions:
        q_id = question["id"]
        q_text = question["question"]
        q_type = question.get("type", "text")
        required = question.get("required", False)
        
        print(f"\n{q_id.upper()}: {q_text}")
        
        if q_type == "choice" and "options" in question:
            print("Options:")
            for idx, option in enumerate(question["options"], 1):
                print(f"  {idx}. {option}")
            
            while True:
                answer = input("\nYour choice (number or text): ").strip()
                if answer.isdigit():
                    idx = int(answer) - 1
                    if 0 <= idx < len(question["options"]):
                        answers[q_id] = question["options"][idx]
                        break
                elif answer in question["options"]:
                    answers[q_id] = answer
                    break
                elif not required:
                    break
                print("Invalid choice, please try again.")
        else:
            answer = input("Your answer: ").strip()
            if answer or not required:
                answers[q_id] = answer
    
    print("\n✓ Answers collected")
    
    return {"user_answers": answers}


In [9]:
def create_modification_plan(state: ModificationState):
    """
    Create detailed plan for modifications using LLM
    """
    
    original_workflow = state.get("original_workflow", {})
    modification_request = state.get("modification_request", "")
    analysis = state.get("analysis", {})
    user_answers = state.get("user_answers", {})
    
    print("\n🤖 CREATING MODIFICATION PLAN")
    print("="*80)
    
    prompt = f"""
You are creating a detailed modification plan for a workflow.

ORIGINAL WORKFLOW:
{json.dumps(original_workflow, indent=2)}

MODIFICATION REQUEST: {modification_request}

ANALYSIS: {json.dumps(analysis, indent=2)}

USER CLARIFICATIONS: {json.dumps(user_answers, indent=2)}

Create a detailed step-by-step modification plan. Return ONLY valid JSON:
{{
  "modifications": [
    {{
      "step": 1,
      "component": "approval_chain",
      "action": "add_approver",
      "details": {{
        "new_approver": "Department Head",
        "level": 2,
        "insert_position": "before_level_3"
      }},
      "reason": "User requested to add Department Head approval"
    }}
  ],
  "cascading_changes": [
    {{
      "component": "form_schema",
      "change": "Add field for Department Head email",
      "automatic": true
    }},
    {{
      "component": "excel_schema",
      "change": "Add tracking columns for Department Head",
      "automatic": true
    }},
    {{
      "component": "workflow_steps",
      "change": "Insert approval step for Department Head",
      "automatic": true
    }}
  ],
  "verification_needed": ["Check approval sequence is logical"],
  "estimated_impact": "moderate"
}}

Return ONLY the JSON.
"""

    try:
        messages = [
            SystemMessage(content="You create modification plans. Always return valid JSON only."),
            HumanMessage(content=prompt)
        ]
        
        response = model.invoke(messages)
        response_text = response.content.strip()
        
        if response_text.startswith("```json"):
            response_text = response_text.replace("```json", "").replace("```", "").strip()
        
        plan = json.loads(response_text)
        
        print(f"\n✓ Plan created with {len(plan.get('modifications', []))} modifications")
        print(f"✓ Cascading changes: {len(plan.get('cascading_changes', []))}")
        
        print("\n📋 MODIFICATION PLAN:")
        for mod in plan.get("modifications", []):
            print(f"   Step {mod['step']}: {mod['action']} in {mod['component']}")
        
        return {"modification_plan": plan}
        
    except Exception as e:
        print(f"\n⚠️  Plan creation failed: {e}")
        return {"modification_plan": {"modifications": [], "cascading_changes": []}}


def apply_modifications(state: ModificationState):
    """
    Apply the modification plan to create modified workflow
    """
    
    original_workflow = state.get("original_workflow", {}).copy()
    plan = state.get("modification_plan", {})
    
    print("\n🔧 APPLYING MODIFICATIONS")
    print("="*80)
    
    modified_workflow = json.loads(json.dumps(original_workflow))  # Deep copy
    changes_applied = []
    
    for modification in plan.get("modifications", []):
        component = modification.get("component")
        action = modification.get("action")
        details = modification.get("details", {})
        
        print(f"\n✓ Applying: {action} to {component}")
        
        try:
            if component == "approval_chain":
                modified_workflow = modify_approval_chain(modified_workflow, action, details)
                changes_applied.append(f"Modified approval chain: {action}")
            
            elif component == "form_schema":
                modified_workflow = modify_form_schema(modified_workflow, action, details)
                changes_applied.append(f"Modified form schema: {action}")
            
            elif component == "excel_schema":
                modified_workflow = modify_excel_schema(modified_workflow, action, details)
                changes_applied.append(f"Modified Excel schema: {action}")
            
            elif component == "workflow_steps":
                modified_workflow = modify_workflow_steps(modified_workflow, action, details)
                changes_applied.append(f"Modified workflow steps: {action}")
            
            elif component == "notifications":
                modified_workflow = modify_notifications(modified_workflow, action, details)
                changes_applied.append(f"Modified notifications: {action}")
            
        except Exception as e:
            print(f"   ⚠️  Failed to apply: {e}")
            changes_applied.append(f"Failed: {action} to {component}")
    
    # Apply cascading changes
    print("\n🔄 Applying cascading changes...")
    for cascade in plan.get("cascading_changes", []):
        if cascade.get("automatic"):
            print(f"   ✓ {cascade.get('change')}")
    
    # Update metadata
    if "metadata" in modified_workflow:
        modified_workflow["metadata"]["version"] = increment_version(
            modified_workflow["metadata"].get("version", "1.0")
        )
        modified_workflow["metadata"]["last_modified"] = "2024-12-03T00:00:00Z"
        modified_workflow["metadata"]["modification_history"] = modified_workflow.get("metadata", {}).get("modification_history", [])
        modified_workflow["metadata"]["modification_history"].append({
            "timestamp": "2024-12-03T00:00:00Z",
            "changes": changes_applied
        })
    
    print(f"\n✅ Applied {len(changes_applied)} changes successfully")
    
    return {
        "modified_workflow": modified_workflow,
        "changes_applied": changes_applied
    }


def modify_approval_chain(workflow: Dict[str, Any], action: str, details: Dict[str, Any]) -> Dict[str, Any]:
    """
    Modify the approval chain
    """
    
    approval_chain = workflow.get("workflow_analysis", {}).get("approval_chain", [])
    
    if action == "add_approver":
        new_approver = {
            "level": details.get("level", len(approval_chain) + 1),
            "approver_role": details.get("new_approver", "New Approver"),
            "approver_type": "single",
            "source": "from_form",
            "conditions": [f"Level {details.get('level')} approver"],
            "rejection_behavior": details.get("rejection_behavior", "end_workflow"),
            "notification_rules": [],
            "timeout_hours": 48
        }
        
        # Insert at specified position
        insert_pos = details.get("level", len(approval_chain) + 1) - 1
        approval_chain.insert(insert_pos, new_approver)
        
        # Renumber levels
        for idx, approver in enumerate(approval_chain, 1):
            approver["level"] = idx
        
        # Add corresponding form fields
        role_field = details.get("new_approver", "").replace(" ", "_").lower()
        form_questions = workflow.get("microsoft_forms", {}).get("questions", [])
        
        form_questions.extend([
            {
                "id": f"q{len(form_questions) + 1}",
                "field_name": f"{role_field}_name",
                "type": "text",
                "title": f"{details.get('new_approver')} Name",
                "required": True,
                "purpose": "approver_identification"
            },
            {
                "id": f"q{len(form_questions) + 2}",
                "field_name": f"{role_field}_email",
                "type": "email",
                "title": f"{details.get('new_approver')} Email",
                "required": True,
                "purpose": "approver_contact"
            }
        ])
    
    elif action == "remove_approver":
        level_to_remove = details.get("level")
        approval_chain = [a for a in approval_chain if a["level"] != level_to_remove]
        
        # Renumber
        for idx, approver in enumerate(approval_chain, 1):
            approver["level"] = idx
    
    elif action == "reorder":
        # Implement reordering logic
        pass
    
    workflow["workflow_analysis"]["approval_chain"] = approval_chain
    
    return workflow


def modify_form_schema(workflow: Dict[str, Any], action: str, details: Dict[str, Any]) -> Dict[str, Any]:
    """
    Modify form schema
    """
    
    form_questions = workflow.get("microsoft_forms", {}).get("questions", [])
    
    if action == "add_field":
        new_field = {
            "id": f"q{len(form_questions) + 1}",
            "field_name": details.get("field_name", "new_field"),
            "type": details.get("type", "text"),
            "title": details.get("label", "New Field"),
            "required": details.get("required", False),
            "validation": details.get("validation", "")
        }
        form_questions.append(new_field)
    
    elif action == "remove_field":
        field_name = details.get("field_name")
        form_questions = [q for q in form_questions if q.get("field_name") != field_name]
    
    elif action == "modify_field":
        field_name = details.get("field_name")
        for question in form_questions:
            if question.get("field_name") == field_name:
                question.update(details.get("updates", {}))
    
    workflow["microsoft_forms"]["questions"] = form_questions
    
    return workflow


In [10]:
def modify_excel_schema(workflow: Dict[str, Any], action: str, details: Dict[str, Any]) -> Dict[str, Any]:
    """
    Modify Excel tracking schema
    """
    
    excel_columns = workflow.get("excel_tracker", {}).get("columns", [])
    
    if action == "add_column":
        new_column = {
            "name": details.get("column_name", "NewColumn"),
            "type": details.get("type", "text"),
            "description": details.get("description", "")
        }
        excel_columns.append(new_column)
    
    elif action == "remove_column":
        column_name = details.get("column_name")
        excel_columns = [c for c in excel_columns if c.get("name") != column_name]
    
    workflow["excel_tracker"]["columns"] = excel_columns
    
    return workflow

In [11]:
def modify_workflow_steps(workflow: Dict[str, Any], action: str, details: Dict[str, Any]) -> Dict[str, Any]:
    """
    Modify Power Automate workflow steps
    """
    
    steps = workflow.get("power_automate_workflow", {}).get("steps", [])
    
    if action == "add_step":
        new_step = {
            "step_number": details.get("position", len(steps) + 1),
            "step_id": details.get("step_id", f"new_step_{len(steps) + 1}"),
            "name": details.get("name", "New Step"),
            "type": details.get("type", "action"),
            "connector": details.get("connector", ""),
            "operation": details.get("operation", "")
        }
        
        insert_pos = details.get("position", len(steps) + 1) - 1
        steps.insert(insert_pos, new_step)
        
        # Renumber
        for idx, step in enumerate(steps, 1):
            step["step_number"] = idx
    
    elif action == "remove_step":
        step_id = details.get("step_id")
        steps = [s for s in steps if s.get("step_id") != step_id]
        
        # Renumber
        for idx, step in enumerate(steps, 1):
            step["step_number"] = idx
    
    workflow["power_automate_workflow"]["steps"] = steps
    
    return workflow

In [12]:
def modify_notifications(workflow: Dict[str, Any], action: str, details: Dict[str, Any]) -> Dict[str, Any]:
    """
    Modify notification rules
    """
    
    notifications = workflow.get("workflow_analysis", {}).get("notifications", [])
    
    if action == "add_notification":
        new_notification = {
            "trigger": details.get("trigger", ""),
            "recipients": details.get("recipients", []),
            "platform": "Outlook",
            "template": details.get("template", "notification")
        }
        notifications.append(new_notification)
    
    elif action == "modify_notification":
        trigger = details.get("trigger")
        for notif in notifications:
            if notif.get("trigger") == trigger:
                notif.update(details.get("updates", {}))
    
    workflow["workflow_analysis"]["notifications"] = notifications
    
    return workflow

In [13]:
def increment_version(version: str) -> str:
    """
    Increment version number
    """
    parts = version.split(".")
    if len(parts) >= 2:
        parts[-1] = str(int(parts[-1]) + 1)
        return ".".join(parts)
    return version

In [14]:
def validate_modifications(state: ModificationState):
    """
    Validate the modified workflow
    """
    
    modified_workflow = state.get("modified_workflow", {})
    original_workflow = state.get("original_workflow", {})
    
    print("\n🔍 VALIDATING MODIFICATIONS")
    print("="*80)
    
    issues = []
    warnings = []
    
    # Check approval chain integrity
    approval_chain = modified_workflow.get("workflow_analysis", {}).get("approval_chain", [])
    levels = [a["level"] for a in approval_chain]
    expected_levels = list(range(1, len(levels) + 1))
    
    if levels != expected_levels:
        issues.append("Approval chain levels are not sequential")
    
    # Check form fields match approval chain
    form_questions = modified_workflow.get("microsoft_forms", {}).get("questions", [])
    approver_emails_in_form = [q for q in form_questions if "email" in q.get("field_name", "").lower() and q.get("purpose") == "approver_contact"]
    
    if len(approver_emails_in_form) < len(approval_chain):
        warnings.append("Some approvers may not have email fields in the form")
    
    # Check workflow steps
    workflow_steps = modified_workflow.get("power_automate_workflow", {}).get("steps", [])
    if len(workflow_steps) == 0:
        issues.append("No workflow steps defined")
    
    # Validation result
    if not issues:
        print("\n✅ All validations passed")
        if warnings:
            print(f"⚠️  {len(warnings)} warning(s):")
            for w in warnings:
                print(f"   • {w}")
    else:
        print(f"\n❌ {len(issues)} issue(s) found:")
        for i in issues:
            print(f"   • {i}")
    
    return {
        "validation_result": {
            "valid": len(issues) == 0,
            "issues": issues,
            "warnings": warnings
        }
    }

In [15]:
def display_results(state: ModificationState):
    """
    Display modification results
    """
    
    modified_workflow = state.get("modified_workflow", {})
    changes_applied = state.get("changes_applied", [])
    validation_result = state.get("validation_result", {})
    
    print("\n" + "="*80)
    print("✅ WORKFLOW MODIFICATION COMPLETE")
    print("="*80)
    
    print(f"\n📋 Workflow: {modified_workflow.get('metadata', {}).get('workflow_name', 'Unknown')}")
    print(f"📝 Version: {modified_workflow.get('metadata', {}).get('version', 'N/A')}")
    
    print(f"\n🔧 CHANGES APPLIED ({len(changes_applied)}):")
    for change in changes_applied:
        print(f"   ✓ {change}")
    
    if validation_result.get("valid"):
        print("\n✅ Validation: PASSED")
    else:
        print(f"\n❌ Validation: FAILED ({len(validation_result.get('issues', []))} issues)")
    
    print("\n📄 MODIFIED WORKFLOW JSON:")
    print("="*80)
    print(json.dumps(modified_workflow, indent=2))
    
    # Save to file
    output_file = "modified_workflow.json"
    with open(output_file, "w") as f:
        json.dump(modified_workflow, f, indent=2)
    
    print("\n" + "="*80)
    print(f"💾 Saved to: {output_file}")
    print("="*80)
    
    return state

In [16]:
def should_ask_questions(state: ModificationState) -> str:
    """
    Decide if clarification needed
    """
    analysis = state.get("analysis", {})
    
    if analysis.get("requires_clarification", False):
        return "ask"
    else:
        return "proceed"


In [17]:
def build_modification_graph():
    """
    Build the workflow modification graph
    """
    
    graph = StateGraph(ModificationState)
    
    # Add nodes
    graph.add_node("load_workflow", load_existing_workflow)
    graph.add_node("analyze_request", analyze_modification_request)
    graph.add_node("generate_questions", generate_clarifying_questions)
    graph.add_node("collect_clarifications", collect_clarifications)
    graph.add_node("create_plan", create_modification_plan)
    graph.add_node("apply_modifications", apply_modifications)
    graph.add_node("validate", validate_modifications)
    graph.add_node("display", display_results)
    
    # Define flow
    graph.set_entry_point("load_workflow")
    graph.add_edge("load_workflow", "analyze_request")
    
    # Conditional: ask questions or proceed
    graph.add_conditional_edges(
        "analyze_request",
        should_ask_questions,
        {
            "ask": "generate_questions",
            "proceed": "create_plan"
        }
    )
    
    graph.add_edge("generate_questions", "collect_clarifications")
    graph.add_edge("collect_clarifications", "create_plan")
    graph.add_edge("create_plan", "apply_modifications")
    graph.add_edge("apply_modifications", "validate")
    graph.add_edge("validate", "display")
    graph.add_edge("display", END)
    
    return graph.compile()